In [ ]:
class TClosenessAnonymizer():
  
  def __init__(self , df , quasi_identifiers , sensitive_attributes , write_to_file = False ,verbose = 1):
    L_Diverse_Validate(df , quasi_identifiers , sensitive_attributes)
    self.df = df[quasi_identifiers + sensitive_attributes ]
    self.sensitive_attributes = sensitive_attributes
    self.quasi_identifiers = quasi_identifiers 
    self.verbose = verbose
    self.thresholds = None

########### Methods to write pandas dataframe to a file  #########################

  def file_write(self,file_name = 'output.csv',sep_ = ',',encoding_ = 'utf-8'):
    self.df[quasi_identifiers+sensitive_attributes].to_csv(file_name, sep=sep_, encoding=encoding_)

########## Method to perform T Closeness #########################################

  def make_anonymize(self):
    self.define_thresholds()
    t_closeness_rows = self.df.groupby(self.quasi_identifiers).filter(lambda group: self.check_thresholds(group))
    self.df = t_closeness_rows[self.quasi_identifiers + self.sensitive_attributes]

  def define_thresholds(self):
    thresholds = { column : self.df[column].value_counts()/len(self.df)  for column in self.sensitive_attributes}
    self.thresholds = thresholds

  def check_thresholds(self,grp):
    length_cluster = len(grp)
    accept = True
    for column in self.sensitive_attributes:
      grp_thresholds = grp[column].value_counts()/length_cluster
      for element in grp_thresholds.keys():
        accept = accept and (grp_thresholds[element]) + self.t >= self.thresholds[column][element]
    return accept

######### Public method  ########################################################   

  def anonymize(self, t = 0.2):
    if(t>= 1):
       self.t = 0.2
    else:
        self.t = t 
    self.make_anonymize()

    return self.df